# Install and imports

In [1]:
!pip install tensorflow-addons

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 8.1 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.1
    Uninstalling typeguard-4.4.1:
      Successfully uninstalled typeguard-4.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.4.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [2]:
!pip install keras_tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.5 MB/s eta 0:00:00


In [3]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Layer, InputSpec
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import load_model
from tensorflow.keras import losses
from tensorflow import keras
from sklearn.model_selection import train_test_split
import random
from PIL import Image

## Initialize random seed

In [4]:
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

# Upload images to predict tensile strength

In [5]:
# To access the data file from google drive
# If you have you files locally, plesea ignore this code block
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Path of the image
image_path = "/content/drive/MyDrive/Tesis_Salvador/"
image_list = os.listdir(image_path+"2023/") + os.listdir(image_path+"2024/")

## Functions

In [8]:
# Load images from path
def load_images(image_path, image_list):
  images = []
  labels = []

  i =0
  for image_name in image_list:
      if i < 26:
        path = os.path.join(image_path, "2023/"+image_name)
      else:
        path = os.path.join(image_path, "2024/"+image_name)
      image = Image.open(path,)
      image = image.resize((250, 250))

      # Convert to GrayScale
      if image.mode != 'L':
          image = image.convert('L')


      image_array = np.array(image)
      images.append(image_array)


      if image_name.startswith("G0"):
          labels.append(5.72)
      elif image_name.startswith("G2"):
          labels.append(4.31)
      elif image_name.startswith("G3"):
          labels.append(4.18)
      elif image_name.startswith("G5"):
          labels.append(2.72)
      elif image_name.startswith("G7"):
          labels.append(2.67)

      elif image_name.startswith("Gi0"):
          labels.append(4.06)
      elif image_name.startswith("Gi2"):
          labels.append(6.23)
      elif image_name.startswith("Gi3"):
          labels.append(5.22)
      elif image_name.startswith("Gi5"):
          labels.append(4.11)
      elif image_name.startswith("Gi7"):
          labels.append(3.85)

      i+=1


  images = np.array(images)
  images = images.reshape(-1, 250, 250, 1)
  images = tf.convert_to_tensor(images/255, dtype=tf.float32)
  labels = tf.convert_to_tensor(labels, dtype=tf.float32)
  return images, labels

# Custom accuracy metric
def custom_accuracy(y_true, y_pred):
    return K.mean(K.abs(y_true - y_pred) / K.maximum(K.abs(y_true), K.epsilon()), axis=-1)

# To include GrayScale to RGB conversion
@keras.utils.register_keras_serializable()
def preprocess_grayscale_to_rgb(input_image):
    return tf.image.grayscale_to_rgb(input_image)

# Predictions with a given model on input images
def preds(model, images, labels):

  loaded_model = tf.keras.models.load_model(model, custom_objects={'custom_accuracy': custom_accuracy})
  # Predictions
  predictions = loaded_model.predict(images)
  count = 0
  for i in range(len(predictions)):
      error = float(abs(predictions[i] - labels[i]) * 100 / labels[i])
      if error <= 5.00:
          count += 1

  print(f"Hits within 5% error: {count/len(predictions)}")

  count = 0
  for i in range(len(predictions)):
      error = float(abs(predictions[i] - labels[i]) * 100 / labels[i])
      if error <= 10.00:
          count += 1

  print(f"Hits within 10% error: {count/len(predictions)}")
  return predictions


In [9]:
images, labels = load_images(image_path, image_list)
predictions = preds('optimal_hyperparameters_transfer_learning_model.h5', images, labels)

2/2 ━━━━━━━━━━━━━━━━━━━━ 12s 3s/step
Hits within 5% error: 0.8181818181818182
Hits within 10% error: 0.8863636363636364
